In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install chromadb
!pip install catboost
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 6

In [ ]:
import pandas as pd
import numpy as np
import chromadb
import catboost


##Обработка данных

###Переменные

In [ ]:
#категориальные признаки
cat_cols = [
    "channel_code", "city", "city_type",
    "okved", "segment", "start_cluster",
    "index_city_code", "ogrn_month", "ogrn_year",
]

In [ ]:
#Столбцы для обработки некоторомыми фунциями (названия столбцов вынесены, чтобы пользователь мог регулировать обработку + если вы решите изменить название столбцов нужно будет это отобразить только здесь)
select_dict={
    'list_operations': ['cnt_a_oper_1m', 'cnt_b_oper_1m', 'cnt_c_oper_1m'],
    'list_incoming_operations':['cnt_cred_d_oper_1m', 'cnt_cred_e_oper_1m', 'cnt_cred_f_oper_1m', 'cnt_cred_g_oper_1m', 'cnt_cred_h_oper_1m'],#список входящих операций
    'list_outgoing_operations': ['cnt_deb_d_oper_1m', 'cnt_deb_e_oper_1m', 'cnt_deb_f_oper_1m', 'cnt_deb_g_oper_1m', 'cnt_deb_h_oper_1m'],#список исходящих операций
    'list_operations_summi':['sum_a_oper_1m', 'sum_b_oper_1m', 'sum_c_oper_1m']#список сумм A,B,C операций
}

список признаков для удаления

In [ ]:


drop_list=['cnt_deb_g_oper_3m_3',
 'cnt_cred_g_oper_3m_3',
 'cnt_days_cred_g_oper_3m_3',
 'cnt_cred_h_oper_3m_3',
 'ogrn_days_end_quarter_mean',
 'cnt_b_oper_1m_mean',
 'sum_cred_d_oper_1m_mean',
 'cnt_days_cred_g_oper_1m_mean',
 'cnt_deb_d_oper_3m_mean',
 'cnt_cred_d_oper_3m_mean',
 'sum_cred_f_oper_3m_mean',
 'cnt_days_cred_f_oper_3m_mean',
 'cnt_a_oper_1m_sum',
 'cnt_b_oper_1m_sum',
 'sum_of_paym_2m_hide_1',
 'sum_of_paym_6m_hide_1',
 'sum_of_paym_1y_hide_1',
 'city_1',
 'sum_of_paym_1y_hide_2',
 'city_2',
 'city_type_2',
 'cnt_cred_f_oper_3m_2',
 'sum_of_paym_2m_hide_3',
 'sum_of_paym_6m_hide_3',
 'sum_of_paym_1y_hide_3',
 'city_type_3']

###Фунции для заполнения NaN значений

In [ ]:
#самое простое заполнение NaN (не используется)
def simple_fill(df,cat_cols):
  val_cols = df.select_dtypes(include = ["int64", "float64"])
  #считаем среднее значение по столбцам для каждой группы
  #где не смогли посчитать среднее значение заменяем на 0
  df[val_cols.columns] = df[val_cols.columns].fillna(0)
  df.fillna('', inplace=True)
  #df[cat_cols] = df[cat_cols].astype("category")
  return df

#заменяем все NaN в числовых столбцах на среднее значение по столбцам для каждой группы
def smart_fill_na(full_df, num_month):
  #берем только числовые столбцы
  df = full_df.select_dtypes(include = ["int64", "float64"])
  #считаем среднее значение по столбцам для каждой группы
  mean_df = (df.groupby('id').max() + df.groupby('id').min())/2
  #где не смогли посчитать среднее значение заменяем на 0
  mean_df = mean_df.reset_index().fillna(0)
  #дублируем строки несколько раз
  mean_df = mean_df.loc[mean_df.index.repeat(num_month)].reset_index(drop=True)
  #заменяем все NaN на среднее значение по столбцам для каждой группы
  mask_fill_na = df.isna()*mean_df
  #объединяем с исходной таблицой, так чтобы на месте всех NaN стояли средние значения по маске
  new_val_columns = df.fillna(0)+mask_fill_na

  #full_df.update(new_df)
  return new_val_columns



In [ ]:
#заполнение пропущенных строк в тестовом датасете
def fill_spaces(test_df):
  month_count=len(test_df['date'].unique())
  users_count=len(test_df['id'].unique())

  rows = sorted(np.repeat(test_df['id'].unique(),month_count))
  columns = ['id', 'date']# test_df.columns
  dates=list(test_df['date'].unique())*users_count

  # Создание DataFrame с пустыми строками
  empty_df = pd.DataFrame('', index=range(len(rows)), columns=columns)
  empty_df['id']=rows
  empty_df['date']=dates

  # Заполнение пробелов в test_df
  merged_df = pd.merge(empty_df, test_df, on=['id', 'date'], how='left')
  return merged_df

In [ ]:
#Создание векторной бд для поиска схожих строчек и заполнения пропусков
def create_db(good_df_arr):
  ids=[str(x) for x in range(len(good_df_arr))]
  client = chromadb.Client()
  print('good_df_arr:',good_df_arr.shape)

  try:
    client.delete_collection(name="all-my-documents")
  except:
    pass
  collection = client.create_collection("all-my-documents")

  collection.add(
      embeddings=good_df_arr,
      ids=ids
  )
  return collection

#Ищет похожих пользователей в базе данных (эмбединги пользователей состоят из числовых признаков, и чем они ближе - тем ближе пользователи)
def get_simularity_samples(collection,bad_rows_arr):


  print('bad_rows_arr:',bad_rows_arr.shape)
  results = collection.query(
      query_embeddings=bad_rows_arr,
      n_results=2,

  )
  return results


In [ ]:
#заполняет пустые числовые и категориальные значения
def preprocessing_min(df, cat_cols, num_month):
  #заполнение пустых числовых значений
  new_val_columns=smart_fill_na(df, num_month)
  df.update(new_val_columns)

  df[cat_cols] = df[cat_cols].fillna('')
  #заполнение пустых категориальных значений
  df[cat_cols] = df[cat_cols].astype("category")
  return df

#заполнять нулями значения, которые мы не смогли заполнить на прошлом этапе
def preprocessing_simularity_fill(input_df, df_db):
  df=input_df
  df_for_db=df_db

  df = df.drop('id', axis=1)
  #берем только числовые столбцы
  df = df.select_dtypes(include = ["int64", "float64"])

  df_for_db = df_for_db.drop('id', axis=1)
  #берем только числовые столбцы
  df_for_db = df_for_db.select_dtypes(include = ["int64", "float64"])

  #Находим идеальные строчки (сточки без единого NaN)
  df.replace(0, np.nan, inplace=True)
  df_for_db.replace(0, np.nan, inplace=True)

  df_for_fill=df
  good_df=df_for_db.dropna(axis=0)


  #берем заданное количество идеальных строчек
  good_df=good_df.sample(40000)
  good_df_arr=np.array(good_df.values)


  #берем строки, в которых содержится NaN
  bad_rows=df_for_fill[df_for_fill.isna().any(axis=1)]
  bad_rows_arr=bad_rows.fillna(0)
  index_list = bad_rows_arr.index.tolist()
  bad_rows_arr = np.array(bad_rows_arr.values)

  #для каждой строки с пропущеным значением находим схожую
  collection = create_db(good_df_arr)
  results=get_simularity_samples(collection,bad_rows_arr)
  #жадно берем самую ближайшую строку из идеальных строк для каждой плохой строки
  idx=np.array(results['ids'])[:, 0]
  good_values_df = good_df.iloc[idx]


  #Заменяем все NaN значения из плохих строк на значения (стоящие в том же столбце) из идеальных строк. И так для кажой плохой строки
  bad_rows_idxs=bad_rows.index.tolist()
  good_values_df=good_values_df.set_index(pd.Index(bad_rows_idxs))
  good_values_df.replace(0, np.nan, inplace=True)
  good_values_df=good_values_df.dropna(axis=0)
  bad_rows=bad_rows.fillna(good_values_df)


  #Встраивание обработанных строк в исходную таблицу
  input_df.update(bad_rows)
  return input_df

###Фунции для генерации фичей

In [ ]:
#считает среднее значение по всем числовы характеристикам среди месяцев
def meaning(df):
  new_df = df.select_dtypes(include = ["int64", "float64"])
  new_df['id']=df['id']
  new_df = new_df.groupby("id").mean(numeric_only = True)

  #переименовка признаков
  rename_dict={}
  for name in new_df.columns:
    rename_dict[name]=f'{name}_mean'
  new_df.rename(columns=rename_dict, inplace=True)

  return new_df

#создание подпоследовательностей для каждого пользователя
def do_sublist(df, windows):
  # Получаем подсписки
  sublists = [group.tolist() for group in df['start_cluster'].rolling(window=windows)]

  # Убираем первые windows - 1 элементов, так как они содержат NaN и делаем срез шагом windows
  sublists = sublists[windows-1:]
  sublists = sublists[::windows]
  return sublists

#считает очень быстро, но не точно
def light_check_change_clusters(df):
  unique_counts = df.groupby('id')['start_cluster'].nunique()-1
  unique_counts.name='change_clusters'
  return unique_counts

#подсчет сколько раз пользователь менял свой кластер
def check_change_clusters(df, month_to_predict, light_varsion = True):
  #Избавляемся от месяца для предсказаний и сохраняем в список столбец id
  df = df.drop(df[df['date'] == month_to_predict].index)

  if light_varsion : return light_check_change_clusters(df)

  id_pole = list(df['id'])

  #Вызов функции для разбития start_cluster на подсписки
  sublists = do_sublist(df, 2)

  #формируем словарь из кластеров продуктов и булевых значений чтобы понять менел ли пользователь свой продукт при переходе в новый месяц
  dict_start_cluster = {'cluster':[k for group in sublists for k in group], 'change_clusters':[]}

  #Заполняем словарь булевых значений
  for group in sublists:
    for i in range(0, len(group)-1):
      if group[i] != group[i+1]:
        dict_start_cluster['change_clusters'].append(True)
      else:
        dict_start_cluster['change_clusters'].append(False)
    dict_start_cluster['change_clusters'].append(False)

  #Формируем DataFrame из словаря
  df = pd.DataFrame.from_dict(dict_start_cluster)
  df['id'] = id_pole

  df= df.groupby('id')['change_clusters'].sum()

  #df= df['change_clusters'].apply(str).astype("category")
  return df


def rename_columns(old_names, idx_month):
  new_names={}
  for old_name in old_names:
    new_names[old_name]=old_name+"_"+str(idx_month)
  return new_names

#Объедениение всех месяцев в одну строку
def concat_names(df, test_mode, num_month):

  if test_mode:
    start_month=num_month+1
    drop_arr=['date']
    return pd.concat([df[df['date']==f'month_{x}'].set_index('id').drop(drop_arr, axis=1).rename(rename_columns(df.columns, x-num_month), axis=1) for x in range(start_month, start_month+num_month)], axis=1)
  else:
    drop_arr=['date', 'end_cluster']
    start_month=1
    return pd.concat([df[df['date']==f'month_{x}'].set_index('id').drop(drop_arr, axis=1).rename(rename_columns(df.columns, x), axis=1) for x in range(start_month, start_month+num_month)], axis=1)

  return new_df

def drop_useless_columns(new_df, df, num_month):
  new_df = new_df.drop([f'start_cluster_{num_month}'], axis=1)
  return new_df

#смотрит скрывает ли клиент статистику по определеннвм столбцам
def check_hide_payment(df):
  list_of_payment=['sum_of_paym_2m','sum_of_paym_6m','sum_of_paym_1y']
  rename_dict={}
  for x in list_of_payment:
    rename_dict[x]=f'{x}_hide'
  hide_payment = df[list_of_payment]==0
  hide_payment = hide_payment.rename(rename_dict, axis=1)
  #hide_payment['id']=df['id']
  return hide_payment





In [ ]:
#подсчет кол-ва операций A,B,C за месяца и их общее кол-во
def sum_abc(df, list_operations):
  dict_for_column={old:f'{old}_sum' for old in list_operations}
  list_operations2 = list(dict_for_column.keys())
  list_operations2.append('id')
  df = df[list_operations2]


  #Подсчет суммы количества операций за месяца
  new_dff = df.groupby('id').sum().rename(dict_for_column, axis=1)

  #Сумма abc
  new_dff['sum_operations'] = new_dff[new_dff.columns].sum(axis=1)

  return new_dff

def sum_incoming_operations(df, list_incoming_operations):
  #Создание словаря для смены названия колонок
  dict_for_column={old:f'{old}_sum' for old in list_incoming_operations}
  list_incoming_operations2 = list(dict_for_column.keys())
  list_incoming_operations2.append('id')
  df = df[list_incoming_operations2]

  #Подсчет суммы количества входящих операций за 3 месяца
  df = df.groupby('id').sum().rename(dict_for_column, axis=1)

  #Подсчет суммы количества входящих операций D,E,F,G,H
  df['sum_incoming_operations'] = df[df.columns].sum(axis=1)
  return df

def sum_outgoing_operations(df, list_outgoing_operations):
  #Создание словаря для смены названия колонок
  dict_for_column={old:f'{old}_sum' for old in list_outgoing_operations}
  list_outgoing_operations2 = list(dict_for_column.keys())
  list_outgoing_operations2.append('id')
  df = df[list_outgoing_operations2]

  #Подсчет суммы количества входящих операций за 3 месяца
  df = df.groupby('id').sum().rename(dict_for_column, axis=1)

  #Подсчет суммы количества исходящих операций D,E,F,G,H
  df['sum_outgoing_operations'] = df[df.columns].sum(axis=1)
  return df

#Подсчет суммы сумм операций A,B,C
def sum_operations_summi(df, list_operations_summi):
  #Создание словаря для смены названия колонок
  dict_for_column={old:f'{old}_sum' for old in ['sum_a_oper_1m', 'sum_b_oper_1m', 'sum_c_oper_1m']}
  list_incoming_operations2 = list(dict_for_column.keys())
  list_incoming_operations2.append('id')
  df = df[list_incoming_operations2]

  #Подсчет суммы суммы A,B,C операций за 3 месяца
  dff = df.groupby('id').sum().rename(dict_for_column, axis=1)

  #Подсчет суммы суммы A,B,C операций
  dff['sum_operations_summi'] = dff[dff.columns].sum(axis=1)
  return dff

def ratio_of_summi_to_sum_operations(df):
  sum_operations_summi = df['sum_operations_summi'].values
  sum_operations = df['sum_operations'].values

  ratios = np.where(sum_operations != 0, sum_operations_summi / sum_operations, 0)

  return pd.DataFrame(ratios, columns=['ratio_of_summi_to_sum_operations'])

#Отношение входящих к исходящим операциям
def ratio_of_incoming_to_outgoin_operations(df):
  return pd.DataFrame(df.apply(lambda row: row['sum_incoming_operations'] / row['sum_outgoing_operations'] if row['sum_outgoing_operations'] != 0 else 0, axis=1), columns=['ratio_incoming_to_outgoin_operations'])

#Отношение sum_cred_e_oper_1m к sum_deb_h_oper_1m
def ratio_cred_e_to_deb_h(df):
  return pd.DataFrame(df.apply(lambda row: row['sum_cred_e_oper_1m_mean'] * row['sum_deb_h_oper_1m_1'], axis=1), columns=['sum_cred_e_oper_1m_mean * sum_deb_h_oper_1m_1'])

#Отношение min_founderpres к max_founderpres
def ratio_minfound_to_maxfound(df):
  return pd.DataFrame(df.apply(lambda row: row['min_founderpres_1'] / row['max_founderpres_1'] if row['max_founderpres_1'] != 0 else 0, axis=1), columns=['min_founderpres_1 / max_founderpres_1'])

In [ ]:
#выполнение различных операций над некоторыми столбцами
def analise_operations(df, new_df, select_dict):

  #подсчет кол-ва операций A,B,C за месяца и их общее кол-во
  new_df = pd.concat((new_df, sum_abc(df, select_dict['list_operations'])), axis=1)

  #подсчет исходящих кол-ва операций D,E,F,G,H за месяца и их общее кол-во
  new_df = pd.concat((new_df, sum_incoming_operations(df, select_dict['list_incoming_operations'])), axis=1)

  #подсчет входящих кол-ва операций D,E,F,G,H за месяца и их общее кол-во
  new_df = pd.concat((new_df, sum_outgoing_operations(df, select_dict['list_outgoing_operations'])), axis=1)

  #подсчет суммы сумм операций A,B,C за месяца и их общая сумма
  new_df = pd.concat((new_df,sum_operations_summi(df, select_dict['list_operations_summi'])), axis=1)

  #Подсчет отношения входящих к исходящим операциям
  new_df = pd.concat((new_df, ratio_of_incoming_to_outgoin_operations(new_df)), axis=1)

  #Подсчет отношения суммы A,B,C к кол-ву операции A,B,C
  new_df = pd.concat((new_df, ratio_of_summi_to_sum_operations(new_df)), axis=1)

  new_df = pd.concat((new_df, ratio_cred_e_to_deb_h(new_df)), axis=1)

  new_df = pd.concat((new_df, ratio_minfound_to_maxfound(new_df)), axis=1)
  return new_df

In [ ]:
#Отношение минимального к среднему остатку на счету
def ratio_of_min_to_avg_operations(df):
  return pd.DataFrame(df.apply(lambda row: row['balance_amt_min'] / row['balance_amt_avg'] if row['balance_amt_avg'] != 0 else 0, axis=1), columns=['ratio_of_min_to_avg_operations'])

#Отношение максимального к среднему остатку на счету
def ratio_of_max_to_avg_operations(df):
  return pd.DataFrame(df.apply(lambda row: row['balance_amt_max'] / row['balance_amt_avg'] if row['balance_amt_avg'] != 0 else 0, axis=1), columns=['ratio_of_max_to_avg_operations'])



##pipeline добавления всех фитч

pipeline добавление всех фитч

In [ ]:
#фунция, объеденяющая остальные функции обработки и генерации фич
def make_features(df, df_for_db, month_to_predict, cat_cols, select_dict, test_mode=False):
  """
  df - таблица, для которой хотим сгенерировать фичи
  df_for_db - таблица для создания векторной бд
  month_to_predict - месяц, для которого будем предсказывать end_cluster
  cat_cols - катигориальные столбцы
  select_dict - словарь, содержащий в себе названия столбцов для специальной обработки
  test_mode - если True, то обрабатывается тестовый датасет
  """

  #кол-во месяцев для анализа
  num_month=len(list(df['date'].unique()))

  #заполнение пустых значений среднним значением по пользователю: считаем среднее значение по столбцам для каждой группы, где не смогли посчитать среднее значение заменяем на 0
  df = preprocessing_min(df, cat_cols, num_month)

  #смотрит скрывает ли клиент статистику по определеннвм столбцам
  df_1 = check_hide_payment(df)


  #смотрит скрывает ли клиент статистику по определеннвм столбцам
  df = pd.concat((df_1, df), axis=1)

  #Объедениение всех месяцев в одну строку
  new_df = concat_names(df,test_mode,num_month)


  #считает среднее значение по всем числовы характеристикам среди 3 месяцев
  new_df = pd.concat((new_df,meaning(df)), axis=1)

  #подсчет сколько раз пользователь менял свой кластер
  new_df = pd.concat((new_df,check_change_clusters(df, month_to_predict, light_varsion=True)), axis=1)

  #подсчет кол-ва операций A,B,C за 3 месяца и их общее кол-во
  new_df = pd.concat((new_df, sum_abc(df, select_dict['list_operations'])), axis=1)

  #выбрасывание столбца start_cluster для последнего месеца
  new_df = drop_useless_columns(new_df, df, num_month)



  if not test_mode : new_df['end_cluster']=df[df['date']==month_to_predict][['end_cluster', 'id']].set_index('id')

  #удаление лишних признаков
  try:
    new_df = new_df.drop(drop_list, axis=1)
  except:
    print('Столбцы для удаления не найдены в датафрейме. Обновите список drop_list')

  return new_df

# Прогноз на тестовой выборке

In [ ]:
#Ссылка на скачивание весов для модели https://drive.google.com/file/d/1-NWUVVjyqBccwoV3wtn3cWbqS8FraMH2/view?usp=sharing

In [ ]:
path_to_model="/content/drive/MyDrive/purple/catboost_model.bin"

загрузка модели


In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier()
model.load_model(path_to_model)

загрузка датасета

In [ ]:
test_df = pd.read_parquet("/content/drive/MyDrive/purple/test_data.pqt")

test_df = fill_spaces(test_df)

Дополняем фичами

In [ ]:
#Выбор месяца для предсказания end_cluster
month_to_predict_test='month_6'

test_df = make_features(test_df.copy(),_, month_to_predict_test, cat_cols,select_dict, test_mode=True)

In [ ]:
test_df.sample(3)

,balance_amt_avg_1,balance_amt_max_1,balance_amt_min_1,balance_amt_day_avg_1,channel_code_1,city_type_1,index_city_code_1,ogrn_days_end_month_1,ogrn_days_end_quarter_1,ogrn_month_1,...,cnt_days_cred_g_oper_3m_mean,sum_deb_h_oper_3m_mean,cnt_deb_h_oper_3m_mean,cnt_days_deb_h_oper_3m_mean,sum_cred_h_oper_3m_mean,cnt_cred_h_oper_3m_mean,cnt_days_cred_h_oper_3m_mean,change_clusters,cnt_c_oper_1m_sum,sum_operations
id,,,,,,,,,,,,,,,,,,,,,
209261,-0.046367,0.001506,-0.125995,-0.052945,,,,0.788855,1.374898,,...,0.396267,-0.142703,0.547032,0.418798,-0.158176,0.251790,0.396518,1,1.652394,4.211444
271642,0.422721,1.338984,-0.104601,0.424979,channel_code_0,city_type_0,index_city_code_10,-1.301449,0.755427,ogrn_month_2,...,0.384773,1.548530,0.611655,0.918798,2.992839,0.273743,0.740840,0,1.652394,4.998678
276929,-0.010288,-0.023310,-0.070002,-0.009371,channel_code_0,,,0.556599,1.374898,ogrn_month_10,...,0.384773,-0.080078,0.549086,0.529909,-0.197647,0.255642,0.506408,0,1.658624,4.217675


Делаем прогноз

In [ ]:
test_pred = model.predict(test_df, prediction_type="Probability")
test_pred = pd.DataFrame(test_pred, columns=model.classes_)
sorted_classes = sorted(test_pred.columns.to_list())
test_pred_proba_df = test_pred[sorted_classes]

In [ ]:
test_pred_proba_df.shape

(100000, 17)

In [ ]:
test_pred_proba_df.head(2)

,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,0.018041,0.008818,0.020993,0.037692,0.013380,0.000201,0.000907,0.001098,0.013376,0.003551,0.009038,0.000859,0.001408,0.000011,0.002353,0.868265,0.000007
1,0.018799,0.313567,0.001701,0.003511,0.001034,0.000166,0.000317,0.000077,0.002788,0.010166,0.001662,0.000394,0.001029,0.000003,0.001186,0.643402,0.000198


Сохраняем прогноз

In [ ]:
sample_submission_df = pd.read_csv("/content/drive/MyDrive/purple/sample_submission.csv")

In [ ]:
sample_submission_df[sorted_classes] = test_pred_proba_df
sample_submission_df.to_csv("baseline_submission.csv", index=False)